# Project Overview

Introduction:

The provided task involves web scraping, data storage in CSV files, creation of tables in SQLite database, executing SQL queries on the table, and performing similar queries using Pandas SQL on the loaded CSV data.
    
The first CSV file includes the data like - Sno, Movie Name, Director Name(bifurcated into subfields as per number of directors in the movie), Duration, Year, Ratings, and Metascore.
The second CSV file includes the data like - Movie Name, Stars(bifurcated into 4 subfields as per number of stars in the movie), Votes, Genere(bifurcated into 3 subfields as per the number of genere movie belongs), Gross Collection, Popularity, and Certification.
The above task of project will be the phase1. 

Now for phase2: Here, two tables will be created in SQLite database using the columns provided in the CSV files. The data from each CSV file is gonna inserted into the corresponding tables. The subsequent SQL queries are executed on the tables in the SQLite database.
The queries involve retrieving specific details from the tables based on various conditions and sorting orders.

The CSV data is then loaded into Pandas DataFrames. The DataFrames are then used to execute Pandas SQL queries, which involve SQL-like operations on the DataFrames. The queries may include joins, filtering, sorting, and other operations as required. The results of the queries are obtained as Dataframes, which can be used for further analysis and visualization.

# 1

In [1]:
# Importing necessary libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Create a list
df1 = []
df2 = []


# Iterate over all pages
for page_num in range(1,51):

        # Get the content of IMDB page

        url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Extract all movies details and store in a list
        movies = soup.find_all("div", class_ = "lister-item-content")


        # Counter variable
        counter = (page_num - 1)*50 +1

        # Iterate over each movie and extract the required fields
        for movie in movies:
            
                # Extract fields for the first CSV file
                sno = counter
                movie_name = movie.find("h3").find("a").text.strip()
                directors = movie.find_all("p")[2].find_all("a")
                director_names = [director.text.strip() for director in directors]
                duration = movie.find("span", class_ = "runtime").text.strip("min")
                year = movie.find("span", class_ = "lister-item-year").text.strip("()")
                rating = movie.find("div", class_ = "ratings-bar").find("strong").text.strip()
                metascore = movie.find("div", class_ = "ratings-metascore")
                metascore = metascore.find("span").text.strip() if metascore else ""
    
                # Increment the counter
                counter += 1
    
                # Append the extracted data to the first CSV file
                df1.append([sno, movie_name, ",".join(director_names), duration, year, rating, metascore])
    
    
                # Extract details for second CSV file
                movie_name = movie.find("h3").find("a").text.strip()
                stars = movie.find("p", class_ = "").find_all("a")
                star_names = [star.text.strip() for star in stars]
                votes = movie.find("p", class_ = "sort-num_votes-visible").find_all("span")[1].text.strip().replace(",","")
                genre = movie.find("span", class_ = "genre").text.strip().replace("\n","")
                gross_element = movie.find("span", text="Gross:")
                gross = gross_element.find_next_sibling("span").text.strip() if gross_element else ""
                popularity = movie.find("span", class_ = "lister-item-index unbold text-primary").text.strip()
                certification = movie.find("span", class_ = "certificate").text.strip() if movie.find("span", class_ = "certificate") else ""
    
                # Append the extracted data to the second CSV file
                df2.append([movie_name, ",".join(star_names), votes, genre, gross, popularity, certification])
    
# Create DataFrames for both CSV file
df1 = pd.DataFrame(df1, columns = ['Sno', 'Movie Name', 'Director', 'Duration', 'Year', 'Rating', 'Metascore'])
df2 = pd.DataFrame(df2, columns = ['Movie Name', 'Stars', 'Votes', 'Genre', 'Gross collection', 'Popularity', 'Certification'])

# Save to CSV files

df1.to_csv("movie_directors.csv", index = False)
df2.to_csv("movies_genres.csv", index = False)    

In [3]:
print(df1)
print(df2)

       Sno                                         Movie Name  \
0        1                                    The Dark Knight   
1        2      The Lord of the Rings: The Return of the King   
2        3                Spider-Man: Across the Spider-Verse   
3        4                                          Inception   
4        5  The Lord of the Rings: The Fellowship of the Ring   
...    ...                                                ...   
2495  2496                                                Ran   
2496  2497                        Baahubali 2: The Conclusion   
2497  2498                                 Gangs of Wasseypur   
2498  2499                                            Yôjinbô   
2499  2500                                   K.G.F: Chapter 1   

                                               Director Duration  Year Rating  \
0     Christopher Nolan,Christian Bale,Heath Ledger,...     152   2008    9.0   
1     Peter Jackson,Elijah Wood,Viggo Mortensen,Ian ...  

# 2

Now we will create two tables in SQLite and insert the data from the CSV files into the tables

In [4]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect("movies.db")

# Create Table 1
conn.execute(""" 
CREATE TABLE IF NOT EXISTS Table1(
     Sno INTEGER PRIMARY KEY AUTOINCREMENT,
     MovieName TEXT,
     DirectorName TEXT,
     Duration TEXT,
     Genre TEXT,
     Rating REAL)""")

# Insert data into Table 1
df1 = pd.read_csv("movie_directors.csv")
df1.to_sql("Table1", conn, if_exists = "replace", index = False)

# Table 2
conn.execute("""
CREATE TABLE IF NOT EXISTS Table2(
     MovieName TEXT,
     Stars TEXT,
     Votes INTEGER,
     Genre TEXT,
     GrossCollection REAL,
     Popularity INTEGER,
     Certification TEXT)""")

# Insert data
df2 = pd.read_csv("movies_genres.csv")
df2.to_sql("Table2", conn, if_exists = "replace", index = False)

# Commit and close the database connection
conn.commit()
conn.close()

* Now we will start quering the tables using SQL queries

In [5]:
# Lets establish a new connection to the SQLite database
conn = sqlite3.connect("movies.db")

# Queries for table 1

# 1. Let's Display all the details of movies created by directors Christopher and Matt Reeves.
query1 = """
SELECT * FROM Table1
WHERE Director LIKE "%Christopher%" OR Director LIKE "%Matt Reeves%"
"""
result1 = pd.read_sql_query(query1, conn)
print("Query 1:")
print(result1)
print()

# 2. Display all the details of movies with a duration of 140 to 190 minutes.
query2 = """
SELECT * FROM Table1
WHERE Duration >=140 AND Duration <=190"""

result2 = pd.read_sql_query(query2, conn)
print("Query 2:")
print(result2)
print()

# 3. Display all details of movies with ratings above 7 in ascending order.
query3 = """
SELECT * FROM Table1
WHERE Rating > 7
ORDER BY Rating ASC"""

result3 = pd.read_sql_query(query3, conn)
print("Query 3:")
print(result3)
print()

# 4. Display all movie names in descending order.
query4 = """
SELECT DISTINCT `Movie Name` FROM Table1
ORDER BY `Movie Name` DESC"""

result4 = pd.read_sql_query(query4, conn)
print("Query 4:")
print(result4)
print()

# 5. Display movie name starts with 'P' and their rating is greater than 7.
query5 = """
SELECT `Movie Name` FROM Table1
WHERE `Movie Name` LIKE 'P%' AND Rating > 7"""

result5 = pd.read_sql_query(query5, conn)
print("Query 5:")
print(result5)
print()

Query 1:
      Sno             Movie Name  \
0       1        The Dark Knight   
1       4              Inception   
2      22  The Dark Knight Rises   
3      40          Batman Begins   
4      51        The Dark Knight   
..    ...                    ...   
195  2440          Batman Begins   
196  2451        The Dark Knight   
197  2454              Inception   
198  2472  The Dark Knight Rises   
199  2490          Batman Begins   

                                              Director  Duration  Year  \
0    Christopher Nolan,Christian Bale,Heath Ledger,...       152  2008   
1    Christopher Nolan,Leonardo DiCaprio,Joseph Gor...       148  2010   
2    Christopher Nolan,Christian Bale,Tom Hardy,Ann...       164  2012   
3    Christopher Nolan,Christian Bale,Michael Caine...       140  2005   
4    Christopher Nolan,Christian Bale,Heath Ledger,...       152  2008   
..                                                 ...       ...   ...   
195  Christopher Nolan,Christian Bale,Mi

In [6]:
# Queries for table 2

# 1. Display all movie names with star 'Arnold Schwarzenegger' in ascending order.

query6 = """
SELECT  DISTINCT `Movie Name` FROM Table2
WHERE Stars LIKE '%Arnold Schwarzenegger%'
ORDER BY `Movie Name` ASC"""

result6 = pd.read_sql_query(query6, conn)
print("Query 6:")
print(result6)
print()

# 2. Now display all details of the movie with the highest number of votes.
query7 = """
SELECT * FROM Table2
WHERE Votes = (SELECT MAX(Votes) FROM Table2)
LIMIT 1 """

result7 = pd.read_sql_query(query7, conn)
print("Query 7:")
print(result7)
print()

# 3. Now display  movie names with gross colletion in descending order.
query8 = """
SELECT DISTINCT `Movie Name` FROM Table2
ORDER BY `Gross collection` DESC"""

result8 = pd.read_sql_query(query8, conn)
print("Query8:")
print(result8)
print()

# 4. Display the gross collection of movies with the star 'Arnold'.
query9 = """
SELECT DISTINCT `Movie Name`, `Gross collection` FROM Table2
WHERE Stars LIKE '%Arnold%' """

result9 = pd.read_sql_query(query9, conn)
print("Query 9:")
print(result9)
print()

# 5. Dispaly all details of movies with comedy and action genres.
query10 = """
SELECT * FROM Table2
WHERE Genre LIKE '%Comedy%' AND Genre LIKE '%Action%'
"""

result10 = pd.read_sql_query(query10, conn)
print("Query10:")
print(result10)
print()

Query 6:
                   Movie Name
0  Terminator 2: Judgment Day

Query 7:
        Movie Name                                              Stars  \
0  The Dark Knight  Christopher Nolan,Christian Bale,Heath Ledger,...   

     Votes                 Genre Gross collection  Popularity Certification  
0  2739587  Action, Crime, Drama         $534.86M         1.0            UA  

Query8:
                                           Movie Name
0                                   Avengers: Endgame
1                                              Aliens
2                                            Die Hard
3                             Spider-Man: No Way Home
4                                   Top Gun: Maverick
5                                      V for Vendetta
6                                   Kill Bill: Vol. 1
7                              Avengers: Infinity War
8                                                Heat
9                                    K.G.F: Chapter 2
10             

In [7]:
# Subqueries

# 1. Display all details from both tables where movie names are the same
query11 = """
SELECT DISTINCT * FROM Table1
JOIN Table2 ON Table1.`Movie Name` = Table2.`Movie Name` """

result11 = pd.read_sql_query(query11, conn)
print("Query11:")
print(result11)
print()

# 2. Let's display all movie names, Director, ratings, and gross collection where the genre is action.
query12 = """
SELECT DISTINCT Table1.`Movie Name`, Table1.Director, Table1.Rating, Table2.`Gross collection`
FROM Table1
JOIN Table2 ON Table1.`Movie Name` = Table2.`Movie Name`
WHERE Table2.Genre LIKE '%Action%' """

result12 = pd.read_sql_query(query12, conn)
print("Query12:")
print(result12)
print()

# 3. Now query will display all details from both the tables with the highest gross collection.
query13 = """
SELECT DISTINCT *
FROM Table1
JOIN Table2 ON Table1.`Movie Name` = Table2.`Movie Name`
WHERE Table2.`Gross collection` = (SELECT MAX(`Gross collection`) FROM Table2)"""

result13 = pd.read_sql_query(query13, conn)
print("Query13:")
print(result13)
print()

# 4. Now query will display all details from both the tables with the highest rating.
query14 = """SELECT DISTINCT *
FROM Table1
JOIN Table2 ON Table1. `Movie Name` = Table2.`Movie Name`
WHERE Table1.Rating = (SELECT MAX(Rating) FROM Table1)"""

result14 = pd.read_sql_query(query14, conn)
print("Query14:")
print(result14)
print()

# 5. This query will display all the details from both tables with the lowest gross collection and lowest rating.
query15 = """SELECT DISTINCT *
FROM Table1
JOIN Table2 ON Table1.`Movie Name` = Table2.`Movie Name`
WHERE Table1.Rating = (SELECT MIN(Rating) AS MinRating FROM Table1)
 AND Table2.`Gross collection` = (SELECT MIN(`Gross collection`) AS MinGross FROM Table2)"""


result15 = pd.read_sql_query(query15, conn)
print("Query15:")
print(result15)
print()

Query11:
       Sno                                         Movie Name  \
0        1                                    The Dark Knight   
1        2      The Lord of the Rings: The Return of the King   
2        3                Spider-Man: Across the Spider-Verse   
3        4                                          Inception   
4        5  The Lord of the Rings: The Fellowship of the Ring   
...    ...                                                ...   
2495  2496                                                Ran   
2496  2497                        Baahubali 2: The Conclusion   
2497  2498                                 Gangs of Wasseypur   
2498  2499                                            Yôjinbô   
2499  2500                                   K.G.F: Chapter 1   

                                               Director  Duration  Year  \
0     Christopher Nolan,Christian Bale,Heath Ledger,...       152  2008   
1     Peter Jackson,Elijah Wood,Viggo Mortensen,Ian ...     

In [8]:
# Close the connection
conn.close()

* Now we will make solution of above all 15 query using Pandas SQL

In [9]:
! pip install pandasql

In [10]:
from pandasql import sqldf

# Define pandasql function
pysqldf = lambda q: sqldf(q, globals())

# Load CSV files into dataframes
df1 = pd.read_csv("movie_directors.csv")
df2 = pd.read_csv("movies_genres.csv")

# Concatenate the dataframes for subqueries
df_new = pd.merge(df1, df2, on = 'Movie Name')

# Solutions using Pandas SQL

# Q1
query1 = """SELECT * FROM df1
WHERE Director LIKE '%Christopher%' OR Director LIKE '%Matt Reeves%' """

result1 = pysqldf(query1)
print("Query1:")
print(result1)
print()

# Q2
query2 = """SELECT *
FROM df1
WHERE Duration >= 140
AND Duration <= 190"""

result2 = pysqldf(query2)
print("Query 2:")
print(result2)
print()

# Q3
query3 = """SELECT *
FROM df1
WHERE Rating >7
ORDER BY Rating ASC"""

result3 = pysqldf(query3)
print("Query 3:")
print(result3)
print()

# Q4
query4 = """SELECT DISTINCT `Movie Name` FROM df1
ORDER BY `Movie Name` DESC"""

result4 = pysqldf(query4)
print("Query 4:")
print(result4)
print()

# Q5
query5 = """SELECT `Movie Name` FROM df1
WHERE `Movie Name` LIKE 'P%'
AND Rating>7 """

result5 = pysqldf(query5)
print("Query 5:")
print(result5)
print()

# Q6
query6 = """SELECT DISTINCT `Movie Name` FROM df2
WHERE Stars LIKE '%Arnold Schwarzenegger%'
ORDER BY `Movie Name` ASC """

result6 = pysqldf(query6)
print("Query 6:")
print(result6)
print()

# Q7
query7 = """SELECT * FROM df2
WHERE Votes == (SELECT MAX(Votes) FROM df2)"""

result7 = pysqldf(query7)
print("Query 7:")
print(result7)
print()

# Q8
query8 = """SELECT DISTINCT `Movie Name` FROM df2
ORDER BY `Gross collection` DESC """

result8 = pysqldf(query8)
print("Query 8:")
print(result8)
print()

# Q9
query9 = """SELECT DISTINCT `Movie Name`, `Gross collection`
FROM df2 WHERE Stars LIKE '%Arnold%' """

result9 = pysqldf(query9)
print("Query 9 :")
print(result9)
print()

# Q10
query10 = """SELECT * FROM df2
WHERE Genre LIKE '%Comedy%' AND 
Genre LIKE '%Action%' """

result10 = pysqldf(query10)
print("Query 10:")
print(result10)
print()

Query1:
      Sno             Movie Name  \
0       1        The Dark Knight   
1       4              Inception   
2      22  The Dark Knight Rises   
3      40          Batman Begins   
4      51        The Dark Knight   
..    ...                    ...   
195  2440          Batman Begins   
196  2451        The Dark Knight   
197  2454              Inception   
198  2472  The Dark Knight Rises   
199  2490          Batman Begins   

                                              Director  Duration  Year  \
0    Christopher Nolan,Christian Bale,Heath Ledger,...       152  2008   
1    Christopher Nolan,Leonardo DiCaprio,Joseph Gor...       148  2010   
2    Christopher Nolan,Christian Bale,Tom Hardy,Ann...       164  2012   
3    Christopher Nolan,Christian Bale,Michael Caine...       140  2005   
4    Christopher Nolan,Christian Bale,Heath Ledger,...       152  2008   
..                                                 ...       ...   ...   
195  Christopher Nolan,Christian Bale,Mic

In [11]:
# Subqueries

# Q11
query11 = """SELECT DISTINCT * FROM df1
JOIN df2 ON df1. `Movie Name` = df2. `Movie Name` """

result11 = pysqldf(query11)
print("Query 11:")
print(result11)
print()

# Q12
query12 = """SELECT DISTINCT df1. `Movie Name`, df1. Director, df1. Rating, df2. `Gross collection`
FROM df1 JOIN df2 ON df1. `Movie Name` = df2. `Movie Name` 
WHERE df2. Genre LIKE '%Action%'"""

result12 = pysqldf(query12)
print("Query 12:")
print(result12)
print()

# Q13
query13 = """SELECT DISTINCT *
FROM df1
JOIN df2 ON df1. `Movie Name` = df2. `Movie Name`
WHERE df2. `Gross collection` = (SELECT MAX(`Gross collection`)FROM df2) """

result13 = pysqldf(query13)
print("Query 13:")
print(result13)
print()

# Q14
query14 = """SELECT DISTINCT * FROM df1
JOIN df2 ON df1. `Movie Name` = df2. `Movie Name`
WHERE df1. Rating = (SELECT MAX(Rating) FROM df1)"""

result14 = pysqldf(query14)
print("Query14:")
print(result14)
print()

# Q15
query15 = """SELECT DISTINCT * FROM df1
JOIN df2 ON df1. `Movie Name` = df2. `Movie Name`
WHERE df2. `Gross collection` = (SELECT MIN(`Gross collection`) FROM df2)
AND df1. Rating = (SELECT MIN(Rating)FROM df1)"""

result15 = pysqldf(query15)
print("Query 15:")
print(result15)
print()

Query 11:
       Sno                                         Movie Name  \
0        1                                    The Dark Knight   
1        2      The Lord of the Rings: The Return of the King   
2        3                Spider-Man: Across the Spider-Verse   
3        4                                          Inception   
4        5  The Lord of the Rings: The Fellowship of the Ring   
...    ...                                                ...   
2495  2496                                                Ran   
2496  2497                        Baahubali 2: The Conclusion   
2497  2498                                 Gangs of Wasseypur   
2498  2499                                            Yôjinbô   
2499  2500                                   K.G.F: Chapter 1   

                                               Director  Duration  Year  \
0     Christopher Nolan,Christian Bale,Heath Ledger,...       152  2008   
1     Peter Jackson,Elijah Wood,Viggo Mortensen,Ian ...    

Here All queries and subqueries are sorted using Pandasql.